<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Immigration-&amp;-Weather-Pipeline---Data-Engineering-Capstone-Project" data-toc-modified-id="Immigration-&amp;-Weather-Pipeline---Data-Engineering-Capstone-Project-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Immigration &amp; Weather Pipeline - Data Engineering Capstone Project</a></span><ul class="toc-item"><li><span><a href="#Project-Summary" data-toc-modified-id="Project-Summary-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Project Summary</a></span></li><li><span><a href="#Step-1:-Scope-the-Project-and-Gather-Data" data-toc-modified-id="Step-1:-Scope-the-Project-and-Gather-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Step 1: Scope the Project and Gather Data</a></span><ul class="toc-item"><li><span><a href="#Scope" data-toc-modified-id="Scope-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Scope</a></span></li><li><span><a href="#Describe-and-Gather-Data" data-toc-modified-id="Describe-and-Gather-Data-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Describe and Gather Data</a></span></li></ul></li><li><span><a href="#Step-2:-Explore-and-Assess-the-Data" data-toc-modified-id="Step-2:-Explore-and-Assess-the-Data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Step 2: Explore and Assess the Data</a></span><ul class="toc-item"><li><span><a href="#Explore-the-Data" data-toc-modified-id="Explore-the-Data-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Explore the Data</a></span></li><li><span><a href="#Cleaning-Steps" data-toc-modified-id="Cleaning-Steps-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Cleaning Steps</a></span></li></ul></li><li><span><a href="#Step-3:-Define-the-Data-Model" data-toc-modified-id="Step-3:-Define-the-Data-Model-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Step 3: Define the Data Model</a></span><ul class="toc-item"><li><span><a href="#3.1-Conceptual-Data-Model" data-toc-modified-id="3.1-Conceptual-Data-Model-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>3.1 Conceptual Data Model</a></span></li><li><span><a href="#3.2-Mapping-Out-Data-Pipelines" data-toc-modified-id="3.2-Mapping-Out-Data-Pipelines-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>3.2 Mapping Out Data Pipelines</a></span></li></ul></li><li><span><a href="#Step-4:-Run-Pipelines-to-Model-the-Data" data-toc-modified-id="Step-4:-Run-Pipelines-to-Model-the-Data-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Step 4: Run Pipelines to Model the Data</a></span><ul class="toc-item"><li><span><a href="#4.1-Create-the-data-model" data-toc-modified-id="4.1-Create-the-data-model-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>4.1 Create the data model</a></span></li><li><span><a href="#4.2-Data-Quality-Checks" data-toc-modified-id="4.2-Data-Quality-Checks-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>4.2 Data Quality Checks</a></span></li><li><span><a href="#4.3-Data-dictionary" data-toc-modified-id="4.3-Data-dictionary-1.5.3"><span class="toc-item-num">1.5.3&nbsp;&nbsp;</span>4.3 Data dictionary</a></span></li></ul></li><li><span><a href="#Step-5:-Complete-Project-Write-Up" data-toc-modified-id="Step-5:-Complete-Project-Write-Up-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Step 5: Complete Project Write Up</a></span></li></ul></li></ul></div>

# Immigration & Weather Pipeline - Data Engineering Capstone Project

## Project Summary

--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
import os
from pyspark.sql import SparkSession, SQLContext, GroupedData
from pyspark.sql.functions import *

In [ ]:
# Build the Spark session
spark = SparkSession.builder.\
config('spark.jars.packages','saurfang:spark-sas7bdat:2.0.0-s_2.11').enableHiveSupport().getOrCreate()

## Step 1: Scope the Project and Gather Data

### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

We will create an ETL pipeline for the [I94 immigration dataset](https://travel.trade.gov/research/reports/i94/historical/2016.html) and world temperature dataset from [Kaggle](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data). This will analyst to explore the data to determine if immigration is affected by the destination's temperature.

### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

[I94 immigration dataset](https://travel.trade.gov/research/reports/i94/historical/2016.html) was retrieved from from the US National Tourism and Trade Office's site: https://travel.trade.gov/research/reports/i94/historical/2016.html. The data are broken down into files by month. Below are some relevant information contained in the data for this particular project:

* `i94yr`: Year (4 digit number)
* `i94mon`: Month as numerical value (1 or 2 digit number)
* `i94cit`: Reference code of the city of origin (3 digit number)
* `i94port`: Reference code of destination city airport (3 characters)
* `arrdate`: Arrival date (SAS Date)
* `depdate`: Departure date (SAS Date)
* `i94visa`: Reason for immigration

[World temperature dataset](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data) was retrieved from Kaggle: https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data. The data are broken into separate files for different coarseness of areas (e.g. countries vs cities) but we'll specifically only look at cities. The file then specifies some of the following:

* `dt`: Datetime of when temperature was recorded (YYYY-MM-DD)
* `AverageTemperature`:  Average temperature in Celcius of area
* `City`:  Name of city where temperature was recorded
* `Country`:  Name of country where temperature was recorded
* `Latitude`: Latitude of city
* `Longitude`:  Longitude of city

## Step 2: Explore and Assess the Data

### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

In [ ]:
sas_data_dir = '../../../data/18-83510-I94-Data-2016'
# Get all files in the data directory
sas_filenames = [f'{sas_data_dir}/{filename}' for filename in os.listdir(sas_data_dir)]
sas_header_file = f'{sas_data_dir}/I94_SAS_Labels_Descriptions.SAS'


In [ ]:
df_spark = spark.read.format('com.github.saurfang.sas.spark').load(sas_filenames[0])


In [ ]:
df_spark.show(5)

### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





In [ ]:
def clean_i94(i94_spark_df):
        '''
        Cleans i94 Spark dataframe with readable column names and proper date
        and value formats.
        
        Args:
            i94_spark_df: Spark dataframe of data from i94 dataset.
        
        Returns:
            Cleaned Spark dataframe of data from i94 dataset
        '''
        # Format dates
        i94_spark_df_dates = (i94_spark_df
                                .withColumn('arrival_year', col('i94yr').cast('integer'))
                                .withColumn('arrival_month', col('i94mon').cast('integer'))
                                .withColumn('data_base_sas', to_date(lit('01/01/1960'), 'MM/dd/yyyy'))
                                .withColumn('arrival_date', expr('date_add(data_base_sas, arrdate)'))
                                .withColumn('departure_date', expr('date_add(data_base_sas, depdate)'))
                                .drop('arrdate', 'depdate')
        )
        
        # Rename columns and set integer values
        i94_spark_df_clean = (i94_spark_df_dates
                                .withColumn('arrival_port', col('i94port'))                              
                                .withColumn('departure_city', col('i94cit').cast('integer'))
                                .withColumn('mode_of_travel', col('i94mode').cast('integer'))
                                .withColumn('city_of_origin', col('i94cit').cast('integer'))
                                .withColumn('country_of_origin', col('i94res').cast('integer'))
                                .withColumn('age', col('i94bir').cast('integer'))
                                .withColumn('visa_type', col('i94visa').cast('integer'))            
        )
        
        # Final selection of data and order
        i94_spark_df_final = i94_spark_df_clean.select(
                    col('arrival_date'),
                    col('departure_date'),
                    col('arrival_port'),
                    col('city_of_origin'),
                    col('country_of_origin'),
                    col('mode_of_travel'),
                    col('age'),
                    col('visa_type')
                )
        
        return i94_spark_df_final
    

In [ ]:
clean_i94(df_spark).show(5)

## Step 3: Define the Data Model

### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

## Step 4: Run Pipelines to Model the Data 

### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

## Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.